In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Custom function to parse timestamp or return NaN
def parse_timestamp(ts):
    try:
        return pd.to_datetime(ts)
    except ValueError:
        return pd.NaT

# Assuming df_dict_kkn is your dictionary of DataFrames for training
all_data = pd.concat(df_dict_kkn.values())

# Preprocessing
all_data['timestamp'] = all_data['timestamp'].apply(parse_timestamp)
all_data.dropna(subset=['timestamp'], inplace=True)
all_data['Signal Strength'].fillna(-100, inplace=True)
all_data = all_data.groupby(['room', 'MAC Address']).resample('30S', on='timestamp').mean().reset_index()

# Feature Engineering
le = LabelEncoder()
all_data['MAC Address'] = le.fit_transform(all_data['MAC Address'])
features = all_data.pivot_table(index=['timestamp', 'room'], columns='MAC Address', values='Signal Strength').fillna(-100)

# Prepare the dataset
X = features.values
y = features.index.get_level_values('room').values

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and Train KNN Classifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Evaluate the Model
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')